In [ ]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../optimized/data/AsymmOverhanging2BumpsArchitecturalScale/deployed_opt.msh')
driver=l.centralJoint()

mat = elastic_rods.RodMaterial()
#mat.setContour(2000, 0.3, '../../../examples/cross_sections/custom_extrusion_B.obj', stiffAxis=elastic_rods.StiffAxis.D1, simplifyVisualizationMesh = 10)
mat.setContour(20000, 0.3, '../../../examples/cross_sections/190108_section_test_R01.obj', stiffAxis=elastic_rods.StiffAxis.D1, simplifyVisualizationMesh = 10)
l.setMaterial(mat)

#l.swapJointAngleDefinitions() # use obtuse angle instead of acute since the linkage is already opened
l.setPerSegmentRestLength(np.loadtxt('../optimized/data/AsymmOverhanging2BumpsArchitecturalScale/design_parameters.txt'))

jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, np.deg2rad(93), fixedVars=fixedVars)

    
torques = np.loadtxt('data/AsymmOverhanging2BumpsArchitecturalScale/torques_11.txt')
externalForces = np.zeros(l.numDoF())
externalForces[l.jointAngleDoFIndices()] = torques

view = LinkageViewer(l, width=1800, height=1024)
view.setCameraParams(((0.12515109108787825, -3.519781754390383, -0.20146285429421043),
 (0.3617374241899421, -0.42605325102339, 0.8292313689392199),
 (0.0, 0.0, 0.0)))
view.show()

In [ ]:
from write_render_files import writeRenderFiles, writeActuators
directory = 'asymm_overhanging_render_data'
for forceScale in np.linspace(1.0, 0, 51):
    name = 'asymm_overhanging_architectural_force_{}'.format(forceScale)
    with suppress_stdout(): elastic_rods.compute_equilibrium(l, forceScale * externalForces, fixedVars=fixedVars)
    view.update(preserveExisting=True)
    writeRenderFiles(l, directory, name)
    writeActuators(l, np.where(torques > 0.1)[0], directory, name)